Trying to update the Webscraping Indeed Notebook to Python 3

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# API Calls
import requests
# Parse HTML
import bs4
# Handle Dataframes (excel data)
import pandas as pd
# Plotting library
# import matplotlib as plt

## Fetching and Cleaning Indeed Search Data

In [15]:
from indeed_scraper import search_indeed, clean_data, save_data, posting_scraper
from utils import save_data

In [20]:
# Fetch Page Information for Indeed Search
# TODO: Add url encoding so you don't have to do it manually here.
query = "machine+learning"
cities = ["New+York%2C+NY", "San+Francisco%2C+CA", "Boston%2C+MA", "Greenwich%2C+CT"]
max_results_per_city = 100
null_value = "NA"

df = search_indeed(query, cities, max_results_per_city, null_value)
df = clean_data(df)
filename = save_data(df, query, path='./indeed_searches/')
print(filename)

./indeed_searches/machine+learning-2018_11_25-165725


## Getting Data Per Posting Page (WIP)

In [19]:
# # Getting one posting worth of data
# url = data.loc[:, 'url'].values[0]
# html = requests.get(url).text
# soups = bs4.BeautifulSoup(html, "html.parser")
# # Print out job description as one srting
# main_content = soups.find('div', {'class': "jobsearch-JobComponent icl-u-xs-mt--sm jobsearch-JobComponent-bottomDivider"})
# job_description = soups.find('div', {'class': "jobsearch-JobComponent-description icl-u-xs-mt--md"})
# # job_description.get_text("  ", strip=True).strip()

In [34]:
# # Print out job description as one srting
# main_content = soups.find('div', {'class': "jobsearch-JobComponent icl-u-xs-mt--sm jobsearch-JobComponent-bottomDivider"})
# job_description = soups.find('div', {'class': "jobsearch-JobComponent-description icl-u-xs-mt--md"})
# # job_description.get_text("  ", strip=True).strip()

In [21]:
## Getting Data Per Posting Page (WIP)
data = pd.read_csv(f"{filename}.csv", index_col=0)
desc_dataframe = posting_scraper(data, filename)
data['desc'] = desc_dataframe['desc']

data.to_csv(f'{filename}.csv', sep=',', encoding='utf-8')

## Count Vectorizer on File
Seeing most popular words in job posting.

In [22]:
# filename = "machine+learning-2018_11_24-185749"
job_info = pd.read_csv(f'{filename}.csv', index_col=0)

In [23]:
descriptions = []
for txtfile in job_info['desc']:
    with open(txtfile, 'r', encoding='utf-8') as the_file:
        descriptions.append(the_file.read().lower())

In [24]:
# https://medium.com/@cristhianboujon/how-to-list-the-most-common-words-from-text-corpus-using-scikit-learn-dad4d0cab41d
def get_top_n_words(corpus, stop_words=None):
    """
    List the top n words in a vocabulary according to occurrence in a text corpus.
    
    get_top_n_words(["I love Python", "Python is a language programming", "Hello world", "I love the world"]) -> 
    [('python', 2),
     ('world', 2),
     ('love', 2),
     ('hello', 1),
     ('is', 1),
     ('programming', 1),
     ('the', 1),
     ('language', 1)]
    """
    vectorizer = CountVectorizer(stop_words=stop_words)
    X = vectorizer.fit_transform(corpus)
    feature_names = vectorizer.get_feature_names()
    sum_words = X.sum(axis=0).tolist()[0]
    words_freq = zip(feature_names, sum_words)
    words_freq =sorted(words_freq, key = lambda x: -x[1])
    return words_freq

In [53]:
%autoreload 2
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from more_stop_words import more_stop_words
custom_stop_words = ENGLISH_STOP_WORDS.union(more_stop_words)
# from sklearn.model_selection import train_test_split

In [54]:
# print(descriptions)
vectorizer = CountVectorizer(stop_words=custom_stop_words)
X = vectorizer.fit_transform(descriptions)

# Summing words along columns to find total amount of occurences per word.
feature_names = vectorizer.get_feature_names()
sum_words = X.sum(axis=0).tolist()[0]
words_freq = zip(feature_names, sum_words)
words_freq =sorted(words_freq, key = lambda x: -x[1])

In [55]:
final_map = [word for word in words_freq if word[1] > 0]
# print(words_freq[:20])
final_map[:50]

[('python', 379),
 ('deep', 348),
 ('analytics', 323),
 ('language', 310),
 ('ml', 304),
 ('google', 283),
 ('ai', 279),
 ('processing', 230),
 ('statistical', 203),
 ('natural', 182),
 ('modeling', 172),
 ('java', 170),
 ('statistics', 165),
 ('scientists', 159),
 ('artificial', 157),
 ('tensorflow', 156),
 ('font', 150),
 ('quantitative', 143),
 ('vision', 139),
 ('advanced', 129),
 ('financial', 125),
 ('applied', 122),
 ('phd', 122),
 ('marketing', 119),
 ('analytical', 116),
 ('scientist', 112),
 ('model', 108),
 ('frameworks', 105),
 ('york', 104),
 ('spark', 103),
 ('sql', 103),
 ('training', 102),
 ('mathematics', 99),
 ('computational', 98),
 ('optimization', 97),
 ('nlp', 96),
 ('strategy', 96),
 ('mining', 94),
 ('sets', 94),
 ('group', 92),
 ('amazon', 91),
 ('cross', 88),
 ('hadoop', 87),
 ('gartner', 85),
 ('ms', 85),
 ('senior', 84),
 ('strategies', 83),
 ('art', 81),
 ('clinical', 81),
 ('education', 81)]

In [68]:
from sklearn.feature_extraction.text import TfidfTransformer
transformer = TfidfTransformer(use_idf=False)
X_tf = TfidfTransformer(use_idf=False).fit_transform(X)

In [69]:
sum_words_tf = X_tf.sum(axis=0).tolist()[0]
feature_names_tf = transformer.get_feature_names()
print(feature_names_tf)
words_freq_tf  = zip(feature_names_tf, sum_words_tf)


AttributeError: 'TfidfTransformer' object has no attribute 'get_feature_names'

Next Steps are to sort these words into categories (job, technologies, companies)